In [27]:
import os
path = '/Users/jeebanswain/Documents/Smart Hiring/Hiring'
os.chdir(path)

In [28]:
import textract

In [29]:
#Loading all the libraries
import pyPdf
import os
import pandas as pd
import string
import numpy as np
from nltk.corpus import stopwords
import xgboost as xgb

In [38]:
pd.set_option('display.max_colwidth', 50)

In [31]:
path = '/Users/jeebanswain/Documents/Smart Hiring/Hiring'
all_resumes =[]
domain_name = []
wd = os.listdir(path)[1:]
for subdir in wd:
    subdir_path = path+'/'+subdir
    all_files = os.listdir(subdir_path)[1:]
    for fileName in all_files:
        file_path = subdir_path +'/'+ fileName
        all_text = textract.process(file_path)
        domain_name.append(subdir)
        all_resumes.append(all_text)

In [34]:
resume_df = pd.DataFrame()
resume_df['CV'] = all_resumes
resume_df['Domain_Name'] = domain_name
resume_df['Domain_Name'] = resume_df['Domain_Name'].map({'Engineering Manager': 0, 'Middleware': 1,'UI': 2})
print resume_df.shape
resume_df.head()

(155, 2)


CV  \
0  \nSumeet Mishra,\nMantri Elite Apts,                                               Mobile:\n+91-9980066558\nBannerghatta Road,                                E-mail:\nsumeet.mishra@gmail.com\nBangalore - 560 076\nsumeet_recdgp@yahoo.com\n\n\n\nExperience Summary\n    > 11.2 years of experience in product development and sustenance using\n      C/C++ as programming languages on UNIX based platforms.\n    > Worked in various development roles from developer to an Engineering\n      Manager.\n    > Experienced in managing feature teams with both developers (including\n      performance engineers) and QA engineers.\n    > Experienced in working with Agile Methodology (Scrum) as a Scrum\n      Master.\n    > Currently working as Engineering Manager at Intel Security (McAfee).\n\n|Organization         |From    |To       |\n|Intel Security       |Sep-2016|Till date|\n|(McAfee)             |        |         |\n|Oracle India Pvt.    |Jun-2010|Aug-2016 |\n|Ltd.                 |        |         |\n|Nokia Siemens        |Apr-2008|May-2010 |\n|Networks             |        |         |\n|Wipro Technologies   |Aug-2005|Mar-2008 |\n\nAcademics:\n|Qualification       |Year                |University                        |\n|B.E (Computer       |2001-2005           |National Institute of Technology  |\n|Science &           |                    |(NIT), Durgapur                   |\n|Engineering) with   |                    |                                  |\n|77.1%               |                    |                                  |\n|HSC with 79.2%      |2001                |Firdaus Amrut High School,        |\n|                    |                    |Ahmedabad                         |\n|SSC with 83.23%     |1999                |Firdaus Amrut High School,        |\n|                    |                    |Ahmedabad                         |\n\nTechnical Skills:\n|Languages  |C, C++ with Data Structures,        |\n|           |Algorithms & Multithreading. Working|\n|           |knowledge of Core Java.(with MySQL, |\n|           |Oracle RDBMS)                       |\n|OS         |Solaris, Linux                      |\n|Tools      |Clearcase, Purify, Valgrind,        |\n|           |Klocworks, GDB, DBX, SVN, JIRA,     |\n|           |VersionOne, BugZilla                |\n|Protocols  |TCP/IP, HTTP, HTTPS, SIP, Diameter, |\n|           |SOAP, REST                          |\n\n\nWork Experience Details:\n   1. Engineering Manager - Intel Security - McAfee Web Gateway (Sep  2016 -\n      Till date)\n    . Currently working as an Engineering manager of a feature team of 8\n      engineers (5 developers & 3 QA engineers) in McAfee Web Gateway team.\n    . Responsibilities include project management, feature design and\n      development using Agile, people management.\n    . Working on protocols like HTTP, HTTPS, FTP etc., with C++ and Java as\n      the programming languages on multiple (Windows and Unix) platforms.\n\n\n\n   2. Software Development Manager - Oracle SDM Team (June 2015 - Aug 2016)\n    . Worked as an engineering manager of software development for the\n      Subscriber Data Management (SDM) team.\n    . SDM is a place holder of subscriber information and can be used in\n      2G/3G/4G networks.\n    . Created from scratch a team of 8 engineers(4 Developers + 4 QA)\n    . Established and implemented efficient SDLC processes and fostered the\n      transition to an agile methodology.\n    . Delivered 1 major release, and several patches on-time, meeting\n      aggressive deadlines.\n    . Was involved in project management, driving R&D efficiency\n      initiatives, handling multi-site teams, site transition, Agile\n      methodology, mentoring, product training, customer interfacing &\n      emergency handling, feature design and development, bug fixing,\n      interfacing with product managers and architects.\n    . Was active participant in technical design and code review(s).\n    . Worked on multiple telec

In [35]:
import re
stops = set(stopwords.words('english'))
clean_rows = []
for rows in resume_df.CV:
    rows = rows.lower()
    rows = re.sub('[^A-Za-z]+', ' ', rows)
    rows = ' '.join([word for word in rows.split() if word not in stops])
    rows=rows.strip()
    clean_rows.append(rows)
resume_df['cv_cleaned'] = clean_rows

In [39]:
resume_df.head()

,CV,Domain_Name,cv_cleaned
0,"\nSumeet Mishra,\nMantri Elite Apts, ...",0,sumeet mishra mantri elite apts mobile bannerg...
1,SEARCH SOLUTIONS BANGALORE\n\nSatya Prakash Pa...,0,search solutions bangalore satya prakash pande...
2,\nName: TOSHIYANGER WALLING\nCurren...,0,name toshiyanger walling current address first...
3,\n ...,0,kaushik kumar bar email kaushikbar gmail com m...
4,\n\n\n\n SUMIT ...,0,sumit shekhar personal details date birth cont...


In [41]:
#creating tfidf 
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 1000, \
                            #ngram_range= (1,2) 
                            ) 

train_test_tdidf_features = vectorizer.fit_transform(resume_df.cv_cleaned)
train_test_tdidf_features = train_test_tdidf_features.toarray()

df_train_test_tfidf_features = pd.DataFrame(train_test_tdidf_features)
df_train_test_tfidf_features.columns = vectorizer.get_feature_names()

In [10]:
df_train_test_tfidf_features.columns

Index([        u'ability',            u'able',        u'abstract',
              u'academic',          u'access', u'accomplishments',
               u'account',         u'achieve',    u'achievements',
                u'across', 
       ...
                 u'would',           u'write',         u'writing',
               u'written',             u'www',             u'xml',
                    u'xp',           u'yahoo',            u'year',
                 u'years'],
      dtype='object', length=1000)

In [42]:
d_train = xgb.DMatrix(df_train_test_tfidf_features, label=resume_df.Domain_Name)

params = {}
#params['objective'] = 'binary:logistic'
params['objective'] = 'multi:softmax'
#params['objective'] = 'multi:softprob'
params['eval_metric'] = 'merror'
params['eta'] = 0.3
params['max_depth'] = 20
params["subsample"] = 0.7
params['num_class'] = 3 # Three Classes
params["min_child_weight"] = 1
params["colsample_bytree"] = 0.65

model = xgb.cv(dtrain=d_train,nfold=6,params=params,num_boost_round=100)
model = xgb.train(dtrain=d_train,params=params,num_boost_round=6)

[0]	cv-test-merror:0.300000+0.060000	cv-train-merror:0.136000+0.030288
[1]	cv-test-merror:0.306667+0.088443	cv-train-merror:0.076000+0.013663
[2]	cv-test-merror:0.280000+0.056569	cv-train-merror:0.049333+0.015606
[3]	cv-test-merror:0.266667+0.078881	cv-train-merror:0.029333+0.005963
[4]	cv-test-merror:0.266667+0.075425	cv-train-merror:0.016000+0.004619
[5]	cv-test-merror:0.253333+0.075425	cv-train-merror:0.012000+0.004000
[6]	cv-test-merror:0.246667+0.070868	cv-train-merror:0.010667+0.003771
[7]	cv-test-merror:0.273333+0.084591	cv-train-merror:0.009333+0.002981
[8]	cv-test-merror:0.293333+0.097068	cv-train-merror:0.006667+0.005497
[9]	cv-test-merror:0.286667+0.096379	cv-train-merror:0.005333+0.003771
[10]	cv-test-merror:0.293333+0.091409	cv-train-merror:0.005333+0.003771
[11]	cv-test-merror:0.266667+0.097068	cv-train-merror:0.005333+0.003771
[12]	cv-test-merror:0.266667+0.082192	cv-train-merror:0.006667+0.005497
[13]	cv-test-merror:0.266667+0.088443	cv-train-merror:0.005333+0.003771
[1

In [43]:
path = '/Users/jeebanswain/Documents/Smart Hiring/Validation/'
val_resumes =[]
val_domain_name = []
wd = os.listdir(path)[1:]
for subdir in wd:
    subdir_path = path+'/'+subdir
    all_files = os.listdir(subdir_path)[1:]
    for fileName in all_files:
        file_path = subdir_path +'/'+ fileName
        all_text = textract.process(file_path)
        val_domain_name.append(subdir)
        val_resumes.append(all_text)

In [44]:
resume_val_df = pd.DataFrame()
resume_val_df['CV'] = val_resumes
resume_val_df['Domain_Name'] = val_domain_name
print resume_val_df.shape

(20, 2)


In [45]:
resume_val_df

,CV,Domain_Name
0,Curriculum Vitae\n\n (+91) 9739197133\nuserdh...,Engineering Manager
1,Hemanta Pathak; Bangalore; India\nhemantap@gma...,Engineering Manager
2,\n JALAGANDESWARI GANAPATHY\n ...,Engineering Manager
3,KARTHIK ARJUNAN | arjunankarthik@gmail.com| Mo...,Engineering Manager
4,\n NEHA NIGAM\...,Engineering Manager
5,"Madhusoodanan C L,\nMember of Technical Staff ...",Middleware
6,Pankaj Chaudhary\nMobile: 9971493997\n\nEmail-...,Middleware
7,MOHAMED FAZIL\n\nEmail:youthfaz@gmail.com\nMob...,Middleware
8,Mukul Ashok Gundawar\n\nmukul.gundawar@aol.in\...,Middleware
9,\n\n\n\n\n\n\n\n\n\n\n\nTECHNICAL SKILLS\n\nOp...,Middleware


In [46]:
import re
stops = set(stopwords.words('english'))
clean_rows = []
for rows in resume_val_df.CV:
    rows = rows.lower()
    rows = re.sub('[^A-Za-z]+', ' ', rows)
    rows = ' '.join([word for word in rows.split() if word not in stops])
    rows=rows.strip()
    clean_rows.append(rows)
resume_val_df['cv_cleaned'] = clean_rows

In [47]:
features_test = vectorizer.fit_transform(resume_val_df.CV)
tdidf_features_test = features_test.toarray()

tdidf_features_test = pd.DataFrame(tdidf_features_test)

tdidf_features_test.columns = vectorizer.get_feature_names()

print tdidf_features_test.shape

(20, 1000)


In [17]:
#tdidf_features_test.columns

In [48]:
train_tf_idf = pd.DataFrame()
train_tf_idf['feature'] = df_train_test_tfidf_features.columns
train_tf_idf.index = train_tf_idf.feature
#train_tf_idf.head(5)

In [49]:
test_tfidf = tdidf_features_test.transpose()
test_tfidf['feature'] = test_tfidf.index
#test_tfidf.head()

In [50]:
#test_tfidf_final = pd.concat([train_tf_idf,test_tfidf],axis= 1)
test_tfidf_final = pd.merge(train_tf_idf, test_tfidf, on='feature',how='left')
test_tfidf_final.fillna(0,inplace=True)
del test_tfidf_final['feature']
test_tfidf_final.head()
test_tfidf_final2 = test_tfidf_final.transpose()

In [21]:
#test_tfidf_final2.head()

In [51]:
print test_tfidf_final2.shape


(20, 1000)


In [52]:
#print predicted class
d_test = xgb.DMatrix(test_tfidf_final2)
prediction2 = pd.DataFrame(model.predict(d_test,ntree_limit=6))

In [53]:
resume_val_df['predicted_Domain'] = prediction2

In [54]:
resume_val_df['predicted_Domain'] = resume_val_df['predicted_Domain'].map({0:'Engineering Manager', 1:'Middleware',2:'UI'})

In [55]:
resume_val_df

,CV,Domain_Name,cv_cleaned,predicted_Domain
0,Curriculum Vitae\n\n (+91) 9739197133\nuserdh...,Engineering Manager,curriculum vitae userdhiru gmail com dhirendra...,Middleware
1,Hemanta Pathak; Bangalore; India\nhemantap@gma...,Engineering Manager,hemanta pathak bangalore india hemantap gmail ...,Middleware
2,\n JALAGANDESWARI GANAPATHY\n ...,Engineering Manager,jalagandeswari ganapathy email jalaganapathy g...,Engineering Manager
3,KARTHIK ARJUNAN | arjunankarthik@gmail.com| Mo...,Engineering Manager,karthik arjunan arjunankarthik gmail com mobil...,Middleware
4,\n NEHA NIGAM\...,Engineering Manager,neha nigam rc purva riviera appartment maratha...,Engineering Manager
5,"Madhusoodanan C L,\nMember of Technical Staff ...",Middleware,madhusoodanan c l member technical staff iii r...,Middleware
6,Pankaj Chaudhary\nMobile: 9971493997\n\nEmail-...,Middleware,pankaj chaudhary mobile email id pankajiitbhu ...,Middleware
7,MOHAMED FAZIL\n\nEmail:youthfaz@gmail.com\nMob...,Middleware,mohamed fazil email youthfaz gmail com mobile ...,Middleware
8,Mukul Ashok Gundawar\n\nmukul.gundawar@aol.in\...,Middleware,mukul ashok gundawar mukul gundawar aol synops...,Middleware
9,\n\n\n\n\n\n\n\n\n\n\n\nTECHNICAL SKILLS\n\nOp...,Middleware,technical skills operating system windows mac ...,Middleware
